In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

In [ ]:
!pip install jsonlines datasets git+https://github.com/google-research/bleurt.git evaluate sacrebleu

In [ ]:

import jsonlines
data_PATH = drive_PATH + '/data-main/exp2.1.deen-n128-t1-e0.02.jsonl'
data = []

with jsonlines.open(data_PATH) as f:
    for line in f.iter():
        data.append(line)

In [ ]:
import tqdm
import jsonlines
import sys
import numpy as np

# Eval - BLEURT
from datasets import load_metric
bleurt = load_metric('bleurt', 'BLEURT-20-D3')

In [ ]:
def BLEURT_mbrd(candidates):
  k_samples = len(candidates)
  score_matrix = np.zeros((k_samples, k_samples))

  for j1 in range(k_samples):
    references=[candidates[j1][0] for j2 in range(k_samples)]
    predictions=[candidates[j2][0] for j2 in range(k_samples)]
    scores = bleurt.compute(predictions=predictions, references=references)['scores']
    score_matrix[ : ,j1] = scores

  # remove votes for self
  for j1 in range(k_samples):
    for j2 in range(k_samples):
      if j1 == j2:
        score_matrix[j2,j1] = 0

  sum_scores = np.sum(score_matrix, axis=1)
  index = np.argmax(sum_scores)
  mbr_winner = candidates[index]

  return mbr_winner, index, sum_scores, score_matrix

In [ ]:
dataset_name = 'DEEN'
num_examples = 1000
assert(num_examples == len(data))
num_candidates = 128
dataset = data
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-main/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)